# Obteniendo el número de casos positivos, fallecidos y vacunados contra COVID-19 por semana epidemiológica

En este notebook se busca obtener un dataset con el número de **casos positivos**, **fallecidos** y **totalmente vacunados** tanto a **nivel nacional**, por **grupo etáreo** como por **departamento** de todo el Perú.


## A. Lectura y Procesamiento

En esta sección se leen y preprocesan los dataset de **RawData**. 

El dataset de casos positivos **(positivos_covid.csv)** y vacunados **(TB_VACUNACION_COVID19.csv)** no están incluidos en el repositorio debido a su excesivo tamaño ~170 MB y ~2.3 GB respectivamente. Sin embargo se puede descargar directamente desde la página de Datos Abiertos de COVID-19 de MINSA y colocarlos en la dirección RawData:
- [Casos positivos por COVID-19](https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa)
- [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion) 

Para preservar recursos, no se procesará la información usando **Pandas** en estos datasets. En su lugar se utilizará el paquete **Dask**. El dataset no será un **pandas dataframe** si no un **dask dataframe** el cual está separado en *n* particiones de tipo **pandas dataframe**.

### Cargar Librerías

In [12]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask import delayed
from epiweeks import Week
from dask import delayed

import functions as fn

### Dataset de casos positivos

In [13]:
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')

In [14]:
# Leer el dataset (dask dataframe ddf)
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO']
ddf_cas = dd.read_csv(url_cas, sep = ';', usecols = col_cas, 
                      dtype={'EDAD': 'float64', 'FECHA_RESULTADO': 'float64'})
ddf_cas = ddf_cas.dropna().astype({'EDAD': 'int8', 'FECHA_RESULTADO': 'int32'})

# Convertir la columna "fecha_vacunacion" a formato fecha y obtenener el año y semana epidemiológica
ddf_cas = ddf_cas.assign(fecha_vacunacion = dd.to_datetime(ddf_cas['FECHA_RESULTADO'], 
                                                           format = "%Y%m%d", errors="coerce"))
ddf_cas['fecha_vacunacion'] = ddf_cas['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
ddf_cas[['year','epi_week']] = ddf_cas['fecha_vacunacion'].str.split("W", 1, expand=True)
ddf_cas = ddf_cas.astype({'year': 'int16', 'epi_week': 'int8'})

# Categorizar edad en grupos etáreos
ddf_cas['edad_cat'] = ddf_cas.map_partitions(edad_cat, 'EDAD')

# Drop unused cols
ddf_cas = ddf_cas.drop(['FECHA_RESULTADO', 'fecha_vacunacion', 'EDAD'], axis=1)

### Dataset de fallecidos

In [15]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO', 'EDAD_DECLARADA']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32', 
                                                                     'EDAD_DECLARADA': 'int8'})
# Transformar a formato fecha y convertir a semana epidemiológia y año
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')
df_fal[['year','epi_week']] = df_fal['FECHA_FALLECIMIENTO'].apply(lambda date_name : Week.fromdate(date_name) \
                                                                                         .weektuple()).tolist()
# Categorizar la edad en grupos etáreos
df_fal['edad_cat'] = edad_cat(df_fal, 'EDAD_DECLARADA')

#df_fal['SEXO'].replace({'MASCULINO': 0,'FEMENINO':1}, inplace=True)
#df_fal = df_fal[df_fal['SEXO'].apply(lambda x: str(x).isdigit())]

# Change types and drop unused cols
df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8'}).drop(['FECHA_FALLECIMIENTO', 'EDAD_DECLARADA'], axis=1)

### Dataset de vacunados

Para encontrar el departamento respectivo de cada vacunado se prevee lo siguiente:

1) El dataset de vacunados **(TB_VACUNACION_COVID19.csv)** solamente proporciona información sobre la ubicación del establecimientos de salud (eess) llamado **'id_eess'**.

2) El dataset de los centros de vacunación [(TB_CENTRO_VACUNACION.csv)](/RawData/TB_CENTRO_VACUNACION.csv) puede ser utilizado para hacer *match* del **'id_eess'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica que representa a cada lugar.

3) Finalmente, con el dataset de ubigeo [(TB_UBIGEOS.csv)](/RawData/TB_UBIGEOS.csv) es posible reemplazar a cada **'id_ubigeo'** con la variable **'departamento'** respectiva.

Para el dataset de UBIGEO solo se necesita las columnas **'id_ubigeo'** y **'departamento'**. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas **'id_centro de vacunación'** e **'id_ubigeo'**.

In [16]:
# Leer los datasets de ubigeo y centros de vacunación
url_ubigeo = '../RawData/TB_UBIGEOS.csv'
url_vaccenter = '../RawData/TB_EESS.csv'
ubigeo = pd.read_csv(url_ubigeo, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(url_vaccenter, usecols = ['id_eess','id_ubigeo'])

In [17]:
# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

In [19]:
# Leer el dataset (dask dataframe ddf)
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['fecha_vacunacion','dosis', 'edad', 'id_eess']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'edad': 'float64'})
# Personas con 3 o menos dosis
ddf_vac = ddf_vac[ddf_vac['dosis'] <= 3].dropna() 
# Convertir la columna 'fecha_vacunacion' a formato fecha y obtener el año y semana epidemiológica
ddf_vac = ddf_vac.assign(fecha_vacunacion = dd.to_datetime(ddf_vac['fecha_vacunacion'], 
                                                           format = '%Y%m%d', 
                                                           errors="coerce"))

ddf_vac['epi_date'] = ddf_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
ddf_vac[['year','epi_week']] = ddf_vac['epi_date'].str.split('W', 1, expand=True)
ddf_vac = ddf_vac.astype({'year': 'int16', 'epi_week': 'int8', 'edad': 'int8'})

# Categorizar la variable "edad" en grupos etáreos
ddf_vac['edad_cat'] = ddf_vac.map_partitions(edad_cat, 'edad')

# Encontrar la variable "departamento" usando "id_eess"
ddf_vac = ddf_vac.merge(vaccenter, how='left', on='id_eess')

# Drop unused cols
ddf_vac = ddf_vac.drop(['fecha_vacunacion', 'epi_date', 'edad', 'id_eess'], axis=1)

## B. Total por cada semana epidemiológica

En esta sección se busca obtener el número **total** de **casos positivos**, **fallecidos** y **vacunados (dosis 1,2 y 3)** por cada **año** y **semana epidemiológica** de todo el Perú. 

In [31]:
# Contar el número de CASOS por año y semana epidemiológica
epi_cas = ddf_cas.groupby(['year', 'epi_week']).count().rename(columns = {'edad_cat':'total_casos'}).compute()
del epi_cas['DEPARTAMENTO']

In [32]:
# Contar el número de FALLECIDOS por año y semana epidemiológica
epi_fal = df_fal.groupby(['year', 'epi_week']).count()
epi_fal = epi_fal.rename(columns = {'DEPARTAMENTO':'total_fallecidos'})
del epi_fal['edad_cat']

In [33]:
# Contar el número de VACUNADOS (dosis 1, dosis 2 y dosis 3) por año y semana epidemiológica
""" Nota: No se puede usar multi-index en la función pivot_table para dask, debido a eso se crea la
función 'crosstab4dask'. Aún es necesario buscar en dask funciones para iterar todas las particiones
más facilmente y optimizar el script. TAREA PENDIENTE """
def crosstab4dask(ddf, var_name):
    """ Función que recibe un dask dataframe 'ddf' y una variable de entrada 'var_name' y realiza 
    una crosstab multi-index de dicha variable por 'year' y 'epi_week' """
    
    lst = []  # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        df = ddf.partitions[i].compute()
        lst.append(pd.crosstab(index=[df['year'],df['epi_week']], columns = df[var_name]))

    merged_epivac = pd.concat(lst, axis=1).fillna(0).astype(np.int64).groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [34]:
epi_dose = crosstab4dask(ddf_vac, 'dosis')

In [35]:
# Unir los dataframes de casos positivos, fallecidos y dosis de vacunación por año y semana epidemiológica
epiweeks = pd.concat([epi_cas, epi_fal, epi_dose], axis=1) \
             .fillna(0).astype('int64')

epiweeks.rename(columns={1: 'total_dosis_1', 
                         2: 'total_dosis_2', 
                         3: 'total_dosis_3'}, 
                inplace=True)
                              
epiweeks

total_casos  total_fallecidos  total_dosis_1  total_dosis_2  \
year epi_week                                                                
2020 10                  6                 3              0              0   
     11                 51                 5              0              0   
     12                279                36              0              0   
     13                458                64              0              0   
     14               1993               230              0              0   
...                    ...               ...            ...            ...   
2022 6               66045              1200         563687         146307   
     7               41283               834         365376         340503   
     8               14490               601         308962         556972   
     9                8786               528         267960         451711   
     10               4125               244         158420         233468   

               total_dosis_3  
year epi_week                 
2020 10                    0  
     11                    0  
     12                    0  
     13                    0  
     14                    0  
...                      ...  
2022 6                547538  
     7                485187  
     8                499548  
     9                458574  
     10               263838  

[106 rows x 5 columns]

## C. Por grupo etáreo de todo el Perú

En esta sección se busca obtener el número de **casos positivos**, **fallecidos** y **personas completamente vacunadas** contra COVID-19 por **grupo etáreo**. Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

In [36]:
# Contar el número de CASOS por grupo etáreo por año y semana epidemiológica
age_dic = {0: '0_17_cas', 1:'18_29_cas',  2: '30_39_cas', 3:'40_49_cas',
           4: '50_59_cas', 5:'60_69_cas', 6:'70_79_cas', 7:'80_more_cas'}  # Age dictionary

df_cas_edad = crosstab4dask(ddf_cas, 'edad_cat').rename(age_dic, axis=1)

In [37]:
# Contar el número de FALLECIDOS por grupo etáreo por año y semana epidemiológica
age_dic = {0: '0_17_fal', 1:'18_29_fal',  2: '30_39_fal', 3:'40_49_fal',
           4: '50_59_fal', 5:'60_69_fal', 6:'70_79_fal', 7:'80_more_fal'}  # Age dictionary

df_fal_edad = pd.crosstab(index = [df_fal['year'], df_fal['epi_week']], 
                          columns = df_fal['edad_cat']).rename(age_dic, axis=1)

In [38]:
# Contar el número de TOTALMENTE VACUNADOS (2 dosis) por grupo etáreo por año y semana epidemiológica
age_dic = {0: '0_17_vac', 1:'18_29_vac',  2: '30_39_vac', 3:'40_49_vac',
           4: '50_59_vac', 5:'60_69_vac', 6:'70_79_vac', 7:'80_more_vac'}  # Age dictionary

df_vac_edad = crosstab4dask(ddf_vac, 'edad_cat').rename(age_dic, axis=1)

In [39]:
# Unir los dataframes de grupos etáreos por año y semana epidemiológica
epiweeks_edad = pd.concat([df_cas_edad, df_fal_edad, df_vac_edad], axis=1) \
                  .fillna(0).astype('int64')

epiweeks_edad.rename(columns={1: 'total_dosis_1', 2: 'total_dosis_2', 3: 'total_dosis_3'}, inplace=True)
                             
epiweeks_edad

edad_cat       0_17_cas  18_29_cas  30_39_cas  40_49_cas  50_59_cas  \
year epi_week                                                         
2020 10               1          3          0          0          1   
     11               5         16          8         11          2   
     12              16         69         56         49         38   
     13              12         59        107         95         82   
     14              69        424        429        356        362   
...                 ...        ...        ...        ...        ...   
2022 6             6895      14272      15499      11809       7672   
     7             4121       9173       9535       7276       4805   
     8             1733       3335       3203       2496       1585   
     9             1025       1911       2012       1516       1045   
     10             550        927        882        654        471   

edad_cat       60_69_cas  70_79_cas  80_more_cas  0_17_fal  18_29_fal  ...  \
year epi_week                                                          ...   
2020 10                0          1            0         0          1  ...   
     11                6          3            0         2          0  ...   
     12               34         12            5         0          2  ...   
     13               65         27           11         5          3  ...   
     14              206         97           50         3          5  ...   
...                  ...        ...          ...       ...        ...  ...   
2022 6              5126       2888         1884        18         14  ...   
     7              3303       1864         1206        15         10  ...   
     8              1067        675          396        15         15  ...   
     9               682        375          220         7         19  ...   
     10              328        206          107         4          8  ...   

edad_cat       70_79_fal  80_more_fal  0_17_vac  18_29_vac  30_39_vac  \
year epi_week                                                           
2020 10                0            1         0          0          0   
     11                2            1         0          0          0   
     12                8            6         0          0          0   
     13               16           10         0          0          0   
     14               52           48         0          0          0   
...                  ...          ...       ...        ...        ...   
2022 6               244          557    555738     205497     155235   
     7               176          368    581934     201322     134276   
     8               124          261    762566     214649     142596   
     9               101          199    639318     197668     133373   
     10               61           93    335022     121057      78945   

edad_cat       40_49_vac  50_59_vac  60_69_vac  70_79_vac  80_more_vac  
year epi_week                                                           
2020 10                0          0          0          0            0  
     11                0          0          0          0            0  
     12                0          0          0          0            0  
     13                0          0          0          0            0  
     14                0          0          0          0            0  
...                  ...        ...        ...        ...          ...  
2022 6            159655     105301      46282      20375         9449  
     7            126679      82874      37829      17767         8385  
     8            118133      73097      32112      15093         7236  
     9            104717      55979      24582      15128         7480  
     10            59542      31302      14490      10131         5237  

[106 rows x 24 columns]

## D. Por cada departamento del Perú

En esta sección se busca obtener el número de **casos positivos**, **fallecidos** y **personas completamente vacunadas** contra COVID-19 para cada **departamento**. Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

In [40]:
# Contar el número de CASOS por departamento respecto al año y semana epidemiológica
df_cas_dep = crosstab4dask(ddf_cas, 'DEPARTAMENTO').add_suffix('_cas')

In [41]:
# Contar el número de FALLECIDOS por departamento respecto al año y semana epidemiológica
df_fal_dep = pd.crosstab(index=[df_fal['year'], df_fal['epi_week']],
                         columns=[df_fal['DEPARTAMENTO']],
                         margins = False).add_suffix('_fal')

In [42]:
# Contar el número de personas TOTALMENTE VACUNADAS por departamento respecto al año y semana epidemiológica
df_vac_dep = crosstab4dask(ddf_vac, 'departamento').add_suffix('_vac')

In [43]:
epiweeks_dep = pd.concat([df_cas_dep, df_fal_dep, df_vac_dep], axis = 1).fillna(0)

epiweeks_dep

AMAZONAS_cas  ANCASH_cas  APURIMAC_cas  AREQUIPA_cas  \
year epi_week                                                         
2020 10                   0           0             0             1   
     11                   0           0             0             1   
     12                   0           4             0             2   
     13                   0           2             0            11   
     14                   0          18             1            36   
...                     ...         ...           ...           ...   
2022 6                 1004        2467          1300          6869   
     7                  547        1472           767          3442   
     8                  220         843           466          1317   
     9                  126         527           180           755   
     10                  67         254            89           327   

               ARICA_cas  AYACUCHO_cas  CAJAMARCA_cas  CALLAO_cas  CUSCO_cas  \
year epi_week                                                                  
2020 10                0             0              0           0          0   
     11                0             0              0           1          1   
     12                0             0              0           5          3   
     13                0             1              1          17         13   
     14                0             2              4         239         43   
...                  ...           ...            ...         ...        ...   
2022 6                 0           745           1554        1237       1719   
     7                 0           343            914         865        919   
     8                 0           176            358         153        521   
     9                 0            96            171         151        266   
     10                0            45             54         102        111   

               HUANCAVELICA_cas  ...  LORETO_vac  MADRE DE DIOS_vac  \
year epi_week                    ...                                  
2020 10                       0  ...         0.0                0.0   
     11                       0  ...         0.0                0.0   
     12                       0  ...         0.0                0.0   
     13                       0  ...         0.0                0.0   
     14                       2  ...         0.0                0.0   
...                         ...  ...         ...                ...   
2022 6                      958  ...     52537.0            21991.0   
     7                      423  ...     53818.0            20128.0   
     8                      239  ...     64087.0            22017.0   
     9                      147  ...     54160.0            17938.0   
     10                      72  ...     29336.0            10482.0   

               MOQUEGUA_vac  PASCO_vac  PIURA_vac  PUNO_vac  SAN MARTIN_vac  \
year epi_week                                                                 
2020 10                 0.0        0.0        0.0       0.0             0.0   
     11                 0.0        0.0        0.0       0.0             0.0   
     12                 0.0        0.0        0.0       0.0             0.0   
     13                 0.0        0.0        0.0       0.0             0.0   
     14                 0.0        0.0        0.0       0.0             0.0   
...                     ...        ...        ...       ...             ...   
2022 6               7362.0     7738.0    55053.0   51711.0         33557.0   
     7               8071.0     6421.0    53796.0   53507.0         31290.0   
     8               9343.0     5661.0    60183.0   62970.0         36044.0   
     9               8243.0     5610.0    49088.0   50141.0         28385.0   
     10              5092.0     2312.0    30954.0   25536.0         15039.0   

               TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                 

## E. Dataset Final

En esta sección se unen todos los dataframes obtenidos de **casos positivos**, **fallecidos** y **personas completamente vacunadas** respecto a grupos etáreos, y departamentos por **año** y **semana epidemiológica**. El dataset obtenido *(Data Product 1)* puede encontrarse en el directorio **Data** o clic en [DP1_covid19-peru_x_semanaEpi.csv](Data/DP1_covid19-peru_x_semanaEpi.csv)

In [44]:
epiweeks_final = pd.concat([epiweeks, epiweeks_edad, epiweeks_dep ], axis = 1).fillna(0).astype('int64')
epiweeks_final

total_casos  total_fallecidos  total_dosis_1  total_dosis_2  \
year epi_week                                                                
2020 10                  6                 3              0              0   
     11                 51                 5              0              0   
     12                279                36              0              0   
     13                458                64              0              0   
     14               1993               230              0              0   
...                    ...               ...            ...            ...   
2022 6               66045              1200         563687         146307   
     7               41283               834         365376         340503   
     8               14490               601         308962         556972   
     9                8786               528         267960         451711   
     10               4125               244         158420         233468   

               total_dosis_3  0_17_cas  18_29_cas  30_39_cas  40_49_cas  \
year epi_week                                                             
2020 10                    0         1          3          0          0   
     11                    0         5         16          8         11   
     12                    0        16         69         56         49   
     13                    0        12         59        107         95   
     14                    0        69        424        429        356   
...                      ...       ...        ...        ...        ...   
2022 6                547538      6895      14272      15499      11809   
     7                485187      4121       9173       9535       7276   
     8                499548      1733       3335       3203       2496   
     9                458574      1025       1911       2012       1516   
     10               263838       550        927        882        654   

               50_59_cas  ...  LORETO_vac  MADRE DE DIOS_vac  MOQUEGUA_vac  \
year epi_week             ...                                                
2020 10                1  ...           0                  0             0   
     11                2  ...           0                  0             0   
     12               38  ...           0                  0             0   
     13               82  ...           0                  0             0   
     14              362  ...           0                  0             0   
...                  ...  ...         ...                ...           ...   
2022 6              7672  ...       52537              21991          7362   
     7              4805  ...       53818              20128          8071   
     8              1585  ...       64087              22017          9343   
     9              1045  ...       54160              17938          8243   
     10              471  ...       29336              10482          5092   

               PASCO_vac  PIURA_vac  PUNO_vac  SAN MARTIN_vac  TACNA_vac  \
year epi_week                                                              
2020 10                0          0         0               0          0   
     11                0          0         0               0          0   
     12                0          0         0               0          0   
     13                0          0         0               0          0   
     14                0          0         0               0          0   
...                  ...        ...       ...             ...        ...   
2022 6              7738      55053     51711           33557      24638   
     7              6421      53796     53507           31290      22242   
     8              5661      60183     62970           36044      26039   
     9              5610      49088     50141           28385      22982   
     10             2312      30954     25536           15039      10942   

               TUMBE

In [45]:
epiweeks_final.to_csv('../Data/DP1_covid19-peru_x_semanaEpi.csv')